In [1]:
import pandas as pd
import altair as alt
import datetime

In [2]:
df = pd.read_excel('2014-2020.xlsx')

In [3]:
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12,Unnamed: 13
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN,NaN


## Data preprocessing

I will add some additional columns to agregate them easily in altair.

In [4]:
# Consts
seasons = ['Winter', 'Spring', 'Summer', 'Autumn']
weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Generate year, day, time, season columns
year_column = []
month_column = []
day_column = []
time_column = []
season_column = []
weekday_column = []
month_day_column = []

for i in range(len(df['Час/Дата'])):
  splitted = df['Час/Дата'][i].split('.')
  date = datetime.date(int(splitted[-1]), int(splitted[1]), int(splitted[0].split('-')[1]))

  year_column.append(int(splitted[-1]))
  month_column.append(int(splitted[1]))
  day_column.append(int(splitted[0].split('-')[1]))
  month_day_column.append(date.strftime('%j'))
  time_column.append(int(splitted[0].split('-')[0]))
  season_column.append(seasons[int(splitted[1]) // 3 % 4])
  weekday_column.append(weekdays[date.weekday()])

# Add columns to df
df['Year'] = year_column
df['Month'] = month_column
df['Day'] = day_column
df['Month-Day'] = month_day_column
df['Time'] = time_column
df['Season'] = season_column
df['Weekday'] = weekday_column


In [5]:
# Should make possible to plot > 5,000 rows
alt.data_transformers.enable('csv')

# For task 1
year_aggregated_df = df.groupby('Year').sum()
year_aggregated_df = year_aggregated_df.reset_index()

# For task 2
a = df.get(df['Year'] == 2020)

# For task 3
day_time_df = df.groupby('Time').mean()
day_time_df = day_time_df.reset_index()

# For task 5
weekday_df = df.groupby('Weekday').mean()
weekday_df = weekday_df.reset_index()

## Task 1 - Як змінювалась структура генерації електроенергії за роками?

Спосіб візуалізації був обраний виходячи з того, що потрібно зобразити 3 виміри: час (рік), кількість (МВт) і клас (тип електростанції).

Час, як на мене, досить інтуїтивно лягає на вісь **Х**, тоді як для кількості було обране числове кодування по осі **Y**. Кольором я позначаю різні класи в даному випадку. Я також розглядала варіант лайн чарту, але використаю його в одному з наступних завдань. Тут акцент був на тому, що візуально можна **порівняти** площі кожного класу одне з одним відповідно до року.

In [6]:
# Task 1 - Як змінювалась структура генерації електроенергії за роками?

alt.Chart(year_aggregated_df).transform_fold(
  ['AES', 'TEC', 'VDE', 'TES', 'GES'],
  as_=['source', 'amount']
).mark_bar().encode(
    x = alt.X('Year:N'),
    y = alt.Y('amount:Q', title='Amount (MW)'),
    color = alt.Color('source:N'),
    tooltip = alt.Tooltip(['source:N', 'amount:Q'])
).properties(width = 950, height = 500, title='Як змінювалась структура генерації електроенергії за роками?')

alt.Chart(...)

## Task 2 - Як залежить споживання електроенергії від дня року та години доби?

Знову ж таки, в цьому завданні потрібно зобразити 3 виміри. Оскільки точок по вісі Х досить багато (365) і вони послідовні, я вирішила, що буде доцільно зробити хітмеп і проілюструвати залежність споживання від години дня та дня у році за допомогою кольору (надалі також від синього до оранжевого, в аналогії від "холодного" до "теплого", щоб було легше зчитувати на рівні асоціацій).

Так можна побачити, що під час зими і весни споживання електроенергії сильно відмінне від літа та осені.

Що можна було б покращити - додати вертикальні лінії, аби розбити графік на сезони, щоб візуально легше сприймати і не вираховувати дні :)

In [7]:
# 2 Task - Як залежить споживання електроенергії від дня року та години доби?

alt.Chart(a).mark_rect().encode(
    x = alt.X('Month-Day:Q', title='Day of year'),
    y = alt.Y('Time:Q', title='Time (h)',  sort='descending'),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'blueorange'))
).properties(width = 950, height = 500, title='Як залежить споживання електроенергії від дня року та години доби? (2020 рік)')

alt.Chart(...)

## Task 3 - Як змінюється генерація електроенергії з різних джерел впродовж доби?

Як я наводила описувала в 1 завданні, тут задача є досить схожа, але мені здалось, що буде влучно сфокусуватись на динаміці - на самій "зміні генерації" в часі.

Для цього було вирішено зобразити данні за допомогою лайнчарту, а саме з'єднати точки за допомогою `transform_loess`. Думаю, що даний графік вдало допомає:
- порівняти загально як співвідноситься кількість генерації електроенергії з різних станцій
- переглянути зміни генерації в часі для кожного типу станції

In [8]:
# 3 Task - Як змінюється генерація електроенергії з різних джерел впродовж доби?
alt.Chart(day_time_df).transform_fold(
  ['AES', 'TEC', 'VDE', 'TES', 'GES'],
  as_=['source', 'amount']
).transform_loess('Time', 'amount', groupby=['source']).mark_line().encode(
    x = alt.X('Time:Q', title='Time (h)',  sort='descending'),
    y = alt.Y('amount:Q', scale = alt.Scale(zero = False), title='Amount (MW)'),
    color = alt.Color('source:N')
).properties(width = 950, height = 500, background = '#F9F9F9', padding = 25, title = 'Як змінюється генерація електроенергії з різних джерел впродовж доби?')

alt.Chart(...)

## Task 4 - Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Це завдання є досить схоожим на 2, але розмірніть більша в масштабі, якщо так можна сказати. Мені здалось досить вдалим вирішенням цієї задачі в пункті 2, тому я використала тут той самий підхід.

Спочатку ідея була узагальнити данні за всі роки і знайти середнє для кожного місяця, але я вирішила тут зобразити лише 2020 рік.

Тренд на споживання так само читається як і в 2 завданні :)

In [9]:
# 4 Task - Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

alt.Chart(a).mark_rect().encode(
    x = alt.X('Month:Q', title='Month of year'),
    y = alt.Y('Time:Q', title='Time (h)',  sort='descending'),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'blueorange'))
).properties(width = 950, height = 500, title='Як змінюється споживання електроенергії впродовж доби у розрізі місяців року? (2020 рік)')

alt.Chart(...)

In [10]:
# 4 Task - Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

alt.Chart(a).mark_rect().encode(
    x = alt.X('Season:N', title='', sort=seasons),
    y = alt.Y('Time:Q', title='Time (h)', sort='descending'),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'blueorange'))
).properties(width = 950, height = 500, title='Як змінюється споживання електроенергії впродовж доби у розрізі пір року? (2020 рік)')

alt.Chart(...)

## Task 5  - Як змінюється споживання електроенергії впродовж тижня?

Оскільки тут потрібно зобразити 2 розмірності, я вирішила використати звичайний барчарт.

З інших графіків - сюди б також підійшов лайнчарт якраз для того, щоб простежити зміну, але оскільки в нас всього 7 днів/класів для інтерпритації, це була б просто 1 лінія, яка б дуже далеко одне від одного з'єднувала лінії. Тож барчарт в цій ситуації виглядає мені більш інформативним і таким наче 'wide and bold'.

In [11]:
# 5 Task - Як змінюється споживання електроенергії впродовж тижня?

alt.Chart(weekday_df).mark_bar().encode(
    x = alt.X('Weekday:N'),
    y = alt.Y('CONSUMPTION:Q', scale=alt.Scale(domain=[15000, 18000]), title='Consumption (MW)'),
    tooltip = alt.Tooltip('CONSUMPTION:Q')
).properties(width = 950, height = 500, title='Як змінюється споживання електроенергії впродовж тижня?')

alt.Chart(...)